# Analysing the redundancies of User Stories - Annotations but no US Text

In [2]:
import locale
import os, json
from itertools import combinations

import pandas as pd
import tiktoken
from openpyxl import Workbook
from openpyxl.styles import Alignment, Font
from openpyxl.utils import get_column_letter as utils_get_column_letter

from support_functions.excel_helper import save_to_excel, formatter_ignored_items
from support_functions.load_data import load_datasets_with_annotations as loading
from support_functions.request_handler import process_user_stories, process_user_stories_parallel
from support_functions.data_transformations import convert_annotation_dataset
from support_functions.time_recorder import TimeRecorder
from support_functions.json_validator import validation, chat_gpt_schema_with_annotations

In [3]:
locale.setlocale(locale.LC_ALL, 'de_DE') 
MODEL_VERSION_NAME = os.getenv('MODEL_VERSION')
THREADING = bool(int(os.getenv('THREADING')))
print(f"Threading is used: {THREADING}")
print(MODEL_VERSION_NAME)

Threading is used: True
gpt-3.5-turbo


In [4]:
time_recorders: dict = {}
time_recorder: TimeRecorder = None

### Load all User Stories from the different packages

We use here just the annotations (formatted):
- Triggers
- Targets
- Contains

We do not need:
- POS

Ignored Files as they differer from the common structure [Ref](https://github.com/ace-design/nlp-stories/tree/main?tab=readme-ov-file#note):
- g02-federal-funding
- g13-planningpoker
- g17-cask
- g27-culrepo

Ignored Files as they were used as trainings data
- g05
- g12

Highlighting words with #

In [5]:
not_processed_datasets: dict[str, list] = loading()
print(not_processed_datasets["g02"][0])

{'PID': '#G02#', 'Text': '#G02# As a Data user, I want to have the 12-19-2017 deletions processed.', 'Persona': ['Data user'], 'Action': {'Primary Action': ['have'], 'Secondary Action': ['processed']}, 'Entity': {'Primary Entity': ['12-19-2017 deletions'], 'Secondary Entity': ['']}, 'Benefit': '', 'Triggers': [['Data user', 'have']], 'Targets': [['processed', '12-19-2017 deletions'], ['have', '12-19-2017 deletions']], 'Contains': [], 'Persona POS': {'Persona POS tag': [['NOUN', 'NOUN']], 'Persona POS text': [['Data', 'user']]}, 'Action POS': {'Primary Action POS': {'Primary Action POS tag': [['VERB']], 'Primary Action POS text': [['have']]}, 'Secondary Action POS': {'Secondary Action POS tag': [['VERB']], 'Secondary Action POS text': [['processed']]}}, 'Entity POS': {'Primary Entity POS': {'Primary Entity POS tag': [['NUM', 'NOUN']], 'Primary Entity POS text': [['12-19-2017', 'deletions']]}, 'Secondary Entity POS': {'Secondary Entity POS tag': [[]], 'Secondary Entity POS text': [[]]}}}

### Converting Data to used suited format

In [6]:
datasets, ignored_items = convert_annotation_dataset(not_processed_datasets)

In [26]:
datasets["g12"]

[{'PID': '#G12#',
  'USID': '610',
  'Text': 'As a camp administrator, I want to be able to add campers, so that I can keep track of each individual camper.',
  'Main Part': 'As a camp administrator, I want to be able to add campers, ',
  'Benefit': 'I can keep track of each individual camper',
  'Triggers': {'Main Part': [['camp administrator', 'add']], 'Benefit': []},
  'Targets': {'Main Part': [['add', 'campers']],
   'Benefit': [['keep track', 'individual camper']]},
  'Contains': {'Main Part': [], 'Benefit': []}},
 {'PID': '#G12#',
  'USID': '611',
  'Text': "As a camp administrator, I want to be able to remove campers if they don't attend the camp anymore, so that I can keep the records organized.",
  'Main Part': "As a camp administrator, I want to be able to remove campers if they don't attend the camp anymore, ",
  'Benefit': 'I can keep the records organized',
  'Triggers': {'Main Part': [['camp administrator', 'remove'],
    ['camp', 'campers']],
   'Benefit': []},
  'Target

In [8]:
ignored_items["#G02#"][0]

'PID: #G02#; Text: As a UI designer, I want to redesign the Resources page, so that it matches the new Broker design styles.; Label Type: Contain'

In [9]:
df_ignored_items: pd.DataFrame = None
if True:            
    cols: list[str] =  ["PID", "Text", "Label Type"]
    df_ignored_items = pd.DataFrame(columns=cols)
    current_ignored_data: dict = {}
    curremt_pid: int = 0
    current_text: str = "" 
    for key, set in ignored_items.items():
        for item in set:
            parts = item.split('; ')
            for part in parts:
                key, value = part.split(': ', 1)
                current_ignored_data[key] = value
            df_ignored_items = pd.concat([df_ignored_items, pd.DataFrame([current_ignored_data])], ignore_index = True)               
            df_ignored_items.reset_index()
    save_to_excel(df_ignored_items, formatter_ignored_items, "Incomplete US labelling", os.getenv("OUTPUT_EXCEL_NAME_WITH_ANNOTATIONS"))
print(f"Count of ignored Elements: {df_ignored_items.shape[0]}")
df_ignored_items.head()

Count of ignored Elements: 191


,PID,Text,Label Type
0,#G02#,"As a UI designer, I want to redesign the Resou...",Contain
1,#G02#,"As a UI designer, I want to track the issues t...",Target
2,#G02#,"As a UI designer, I want to track the issues t...",Target
3,#G02#,"As a UI designer, I want to track the issues t...",Target
4,#G02#,"As an agency user, I want to submit my data el...",Contain


### Datatransformation to panda dataframes and numpy arrarys. 

In [10]:
def transform_2_dataframe(data: list):
    rows = []
    for entry in data:
        row = {
            "PID": entry["PID"],
            "USID": entry["USID"],
            "Text": entry["Text"],
            "Main Part": entry["Main Part"],
            "Benefit": entry["Benefit"],
            "Triggers": entry["Triggers"],
            "Targets": entry["Targets"],
            "Contains": entry["Contains"],
        }
        rows.append(row)
    return pd.DataFrame(rows)

In [11]:
# n(n-1) / 2
def transform_pairwise(df: pd.DataFrame):
    rows = []
    for i, j in combinations(range(df.shape[0]), 2):
        row = {
            "First PID": df.iloc[i, 0],
            "First USID": df.iloc[i, 1],
            "First Text": df.iloc[i, 2],
            "First Main Part": df.iloc[i, 3],
            "First Benefit": df.iloc[i, 4],
            "First Triggers": df.iloc[i, 5],
            "First Targets": df.iloc[i, 6],
            "First Contains": df.iloc[i, 7],
            "Second PID": df.iloc[j, 0],
            "Second USID": df.iloc[j, 1],
            "Second Text": df.iloc[j, 2],
            "Second Main Part": df.iloc[j, 3],
            "Second Benefit": df.iloc[j, 4],
            "Second Triggers": df.iloc[j, 5],
            "Second Targets": df.iloc[j, 6],
            "Second Contains": df.iloc[j, 7]
        }
        rows.append(row)
    return pd.DataFrame(rows)

In [12]:
def convert_dataframe_to_json(pair: pd.Series) -> tuple[dict, dict]:
    pair = pair.to_dict()
    us_one: dict = {
        "USID": pair["First USID"],
        "Triggers": pair["First Triggers"],
        "Targets": pair["First Targets"],
        "Contains": pair["First Contains"]
    }
    
    us_two: dict = {
        "PID": pair["Second PID"],
        "USID": pair["Second USID"],
        "Triggers": pair["Second Triggers"],
        "Targets": pair["Second Targets"],
        "Contains": pair["Second Contains"]
    }

    json_one: dict = json.loads(json.dumps(us_one))
    json_two: dict = json.loads(json.dumps(us_two))
    
    return json_one, json_two

In [13]:
def template_request_two_user_stories(
    current_message: list[dict],
    idx: int,
    pairs: pd.DataFrame,
) -> None:
    row = pairs.iloc[idx]
    json_us_one: dict = None
    json_us_two: dict = None
    json_us_one, json_us_two = convert_dataframe_to_json(row)
    request: dict = ### insert from builder
    current_message.append(request)

In [14]:
def sort_threaded_results(to_sort: dict) -> None:
    new_list: list = None
    for _ in to_sort.values():
        new_list = sorted(_, key=lambda x: (int(x['relatedStories'][0]), int(x['relatedStories'][1])))
        _.clear()
        _ += new_list

In [15]:
def json_validator(json_data: dict) -> tuple[bool, str]:
    return validation(json_data, chat_gpt_schema_with_annotations)

### Prepaire Prompting

In [15]:
definition_main_part: str = None 
definition_benefit: str = None
definition_general_redundancy: str = None
definition_main_part_redundancy: str = None
definition_benefit_redundancy: str = None

In [16]:
definition_user_story: str = '''A user story is a semi-structured sentence containing the following:
information: the persona involved in the story, the main part containing the actions that the persona will perform on the system and 
the entities involved in the actions, and optionally a benefit that the persona will receive after having completed these actions. 
The benefit may also include actions and entities. Classically, a user story is expressed in the following form: "As a <persona>, I can <Actions over entities>, so that <benefit>."'''
print(definition_user_story)

A user story is a semi-structured sentence containing the following:
information: the persona involved in the story, the main part containing the actions that the persona will perform on the system and 
the entities involved in the actions, and optionally a benefit that the persona will receive after having completed these actions. 
The benefit may also include actions and entities. Classically, a user story is expressed in the following form: "As a <persona>, I can <Actions over entities>, so that <benefit>."


In [17]:
definition_json_format_pair_user_stories: str = '''
    {
        "relatedStories": [1, 3],
        "redundantMainPart": true, 
        "mainPartRedundancies": [
            partialRedundancy: false,
            fullRedundancy: true,
            redundancies: [
                {
                    "pairsOfTriggerRedundancy": {
                        [["User", "application User"], ["User", "application User"]],
                        ...
                    },
                    "pairsOfTriggerRedundancy": {
                        [["User", "application User"], ["User", "application User"]],
                        ...
                    },
                    "pairsOfTriggerRedundancy": {
                        [["User", "application User"], ["User", "application User"]],
                        ...
                    }
                },
            ],
                
        ],
        "redundantBenefit": true, 
        "benefitRedundancies": [
            // Analog zu oben
        ]
    }
'''

In [18]:
example1: str = None

In [19]:
example2: str = None

In [20]:
example3: str = None

In [21]:
exampl4: str = None

In [22]:
example5: str = None

In [23]:
#Collection of messages -> In future it can be used to create a context aware system
message: list[dict] = []

In [24]:
if True:
    message_text: str = ""
    for key in message:
        message_text += key["content"]
        print(key["content"])
    enc = tiktoken.get_encoding('cl100k_base')
    token_size = enc.encode(message_text)
    print("-" * 3 + "Token" + "-" * 3)
    print(token_size)
    print("-" * 3 + "Sum of Tokens" + "-" * 3)
    # The output is not correct as the result from the online pages differs https://platform.openai.com/tokenizer
    print("The total token sum is: " + locale.format_string("%d", sum(token_size), grouping=True))
    

---Token---
[]
---Sum of Tokens---
The total token sum is: 0


## Data processing for: G02 federal funding

In [25]:
ID_G02: str = "g02"
data_g02: list = datasets[ID_G02]
df_g02: pd.DataFrame = transform_2_dataframe(data_g02)
df_g02_pairs = transform_pairwise(df_g02)

In [26]:
display(df_g02)

,PID,USID,Text,Main Part,Benefit,Triggers,Targets,Contains
0,#G02#,215,"As a Data user, I want to have the 12-19-2017 ...","As a Data user, I want to have the 12-19-2017 ...",,"{'Main Part': [['Data user', 'have']], 'Benefi...","{'Main Part': [['processed', '12-19-2017 delet...","{'Main Part': [], 'Benefit': []}"
1,#G02#,216,"As a UI designer, I want to redesign the Resou...","As a UI designer, I want to redesign the Resou...",it matches the new Broker design styles,"{'Main Part': [['UI designer', 'redesign']], '...","{'Main Part': [['redesign', 'Resources page']]...","{'Main Part': [], 'Benefit': []}"
2,#G02#,217,"As a UI designer, I want to report to the Agen...","As a UI designer, I want to report to the Agen...",they are aware of their contributions to makin...,"{'Main Part': [['UI designer', 'report']], 'Be...","{'Main Part': [['report', 'user testing']], 'B...","{'Main Part': [], 'Benefit': []}"
3,#G02#,218,"As a UI designer, I want to move on to round 2...","As a UI designer, I want to move on to round 2...",I can get approvals from leadership,"{'Main Part': [['UI designer', 'move on'], ['D...","{'Main Part': [['move on', 'round 2']], 'Benef...","{'Main Part': [], 'Benefit': []}"
4,#G02#,220,"As a UI designer, I want to move on to round 3...","As a UI designer, I want to move on to round 3...",I can get approvals from leadership,"{'Main Part': [['UI designer', 'move on'], ['H...","{'Main Part': [['move on', 'round 3']], 'Benef...","{'Main Part': [], 'Benefit': []}"
...,...,...,...,...,...,...,...,...
90,#G02#,308,"As a FABS user, I want to have my validations ...","As a FABS user, I want to have my validations ...",,"{'Main Part': [['FABS user', 'have']], 'Benefi...","{'Main Part': [['have', 'validations']], 'Bene...","{'Main Part': [], 'Benefit': []}"
91,#G02#,309,"As a FABS user, I want to see correct status l...","As a FABS user, I want to see correct status l...",I can quickly see my submission history,"{'Main Part': [['FABS user', 'see'], ['Submiss...","{'Main Part': [['see', 'correct status labels'...","{'Main Part': [], 'Benefit': []}"
92,#G02#,310,"As an agency user, I want to know when the sub...","As an agency user, I want to know when the sub...",I know when the submission starts and ends,"{'Main Part': [['agency user', 'know']], 'Bene...","{'Main Part': [['know', 'submission periods st...","{'Main Part': [], 'Benefit': []}"
93,#G02#,311,"As an agency user, I want a landing page to na...","As an agency user, I want a landing page to na...",I can access both sides of the site,"{'Main Part': [['agency user', 'want'], ['land...","{'Main Part': [['want', 'landing page']], 'Ben...","{'Main Part': [], 'Benefit': []}"


In [27]:
display(df_g02_pairs)

,First PID,First USID,First Text,First Main Part,First Benefit,First Triggers,First Targets,First Contains,Second PID,Second USID,Second Text,Second Main Part,Second Benefit,Second Triggers,Second Targets,Second Contains
0,#G02#,215,"As a Data user, I want to have the 12-19-2017 ...","As a Data user, I want to have the 12-19-2017 ...",,"{'Main Part': [['Data user', 'have']], 'Benefi...","{'Main Part': [['processed', '12-19-2017 delet...","{'Main Part': [], 'Benefit': []}",#G02#,216,"As a UI designer, I want to redesign the Resou...","As a UI designer, I want to redesign the Resou...",it matches the new Broker design styles,"{'Main Part': [['UI designer', 'redesign']], '...","{'Main Part': [['redesign', 'Resources page']]...","{'Main Part': [], 'Benefit': []}"
1,#G02#,215,"As a Data user, I want to have the 12-19-2017 ...","As a Data user, I want to have the 12-19-2017 ...",,"{'Main Part': [['Data user', 'have']], 'Benefi...","{'Main Part': [['processed', '12-19-2017 delet...","{'Main Part': [], 'Benefit': []}",#G02#,217,"As a UI designer, I want to report to the Agen...","As a UI designer, I want to report to the Agen...",they are aware of their contributions to makin...,"{'Main Part': [['UI designer', 'report']], 'Be...","{'Main Part': [['report', 'user testing']], 'B...","{'Main Part': [], 'Benefit': []}"
2,#G02#,215,"As a Data user, I want to have the 12-19-2017 ...","As a Data user, I want to have the 12-19-2017 ...",,"{'Main Part': [['Data user', 'have']], 'Benefi...","{'Main Part': [['processed', '12-19-2017 delet...","{'Main Part': [], 'Benefit': []}",#G02#,218,"As a UI designer, I want to move on to round 2...","As a UI designer, I want to move on to round 2...",I can get approvals from leadership,"{'Main Part': [['UI designer', 'move on'], ['D...","{'Main Part': [['move on', 'round 2']], 'Benef...","{'Main Part': [], 'Benefit': []}"
3,#G02#,215,"As a Data user, I want to have the 12-19-2017 ...","As a Data user, I want to have the 12-19-2017 ...",,"{'Main Part': [['Data user', 'have']], 'Benefi...","{'Main Part': [['processed', '12-19-2017 delet...","{'Main Part': [], 'Benefit': []}",#G02#,220,"As a UI designer, I want to move on to round 3...","As a UI designer, I want to move on to round 3...",I can get approvals from leadership,"{'Main Part': [['UI designer', 'move on'], ['H...","{'Main Part': [['move on', 'round 3']], 'Benef...","{'Main Part': [], 'Benefit': []}"
4,#G02#,215,"As a Data user, I want to have the 12-19-2017 ...","As a Data user, I want to have the 12-19-2017 ...",,"{'Main Part': [['Data user', 'have']], 'Benefi...","{'Main Part': [['processed', '12-19-2017 delet...","{'Main Part': [], 'Benefit': []}",#G02#,221,"As a Developer , I want to be able to log bett...","As a Developer , I want to be able to log bett...",I can troubleshoot issues with particular subm...,"{'Main Part': [['Developer', 'log better']], '...","{'Main Part': [], 'Benefit': [['troubleshoot',...","{'Main Part': [], 'Benefit': []}"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4460,#G02#,309,"As a FABS user, I want to see correct status l...","As a FABS user, I want to see correct status l...",I can quickly see my submission history,"{'Main Part': [['FABS user', 'see'], ['Submiss...","{'Main Part': [['see', 'correct status labels'...","{'Main Part': [], 'Benefit': []}",#G02#,311,"As an agency user, I want a landing page to na...","As an agency user, I want a landing page to na...",I can access both sides of the site,"{'Main Part': [['agency user', 'want'], ['land...","{'Main Part': [['want', 'landing page']], 'Ben...","{'Main Part': [], 'Benefit': []}"
4461,#G02#,309,"As a FABS user, I want to see correct status l...","As a FABS user, I want to see correct status l...",I can quickly see my submission history,"{'Main Part': [['FABS user', 'see'], ['Submiss...","{'Main Part': [['see', 'correct status labels'...","{'Main Part': [], 'Benefit': []}",#G02#,312,"As an agency user, I want to submit my data el...","As an 

### Process and Store the User Stories persistently

In [ ]:
time_recorder = TimeRecorder()
time_recorders[ID_G02] = time_recorder

if THREADING:
    process_user_stories_parallel(index_usid1=1, index_usid2=9, message=message, pairs=df_g02_pairs, key=ID_G02, model_version_name=MODEL_VERSION_NAME, template_request_two_user_stories=template_request_two_user_stories,
                                  sort_threaded_results=sort_threaded_results, json_schema=chat_gpt_schema_with_annotations, redundancy_prefix="with-annoations", time_recorder=time_recorder)
else:
    process_user_stories(index_usid1=1, index_usid2=9, message=message, pairs=df_g02_pairs, key=ID_G02, model_version_name=MODEL_VERSION_NAME, template_request_two_user_stories=template_request_two_user_stories, 
                         json_validation=json_validator, redundancy_prefix="annoations", time_recorder=time_recorder)

Nicht beachtet: G02, G17, G13 

## Save total Speeds

In [ ]:
SHEET_NAME = 'Time Consup. Anlys.S.'

In [ ]:
base_path: str = os.getcwd()
path_to_file = os.path.join(base_path, "results")
path_to_file = os.path.join(path_to_file, "redundancy-model-" + MODEL_VERSION_NAME)

count_runs_per_data_set: dict[str, int] = {}
for key in time_recorders.keys():
        idx = 0
        _ =  f"{idx:02d}_{key}.json"
        while os.path.exists(os.path.join(path_to_file, f"{idx:02d}_{key}.json")):
                idx += 1
        count_runs_per_data_set[key] = idx

entries: list[tuple[int,str,str, int, float, float, float]] = []

entry: tuple[int,str,str, int, float, float, float] = None
for key, value in time_recorders.items():
        entry = (key, count_runs_per_data_set[key], MODEL_VERSION_NAME, str(THREADING).lower(), 
                 value.nanoseconds, value.milliseconds, value.seconds, value.minutes)
        entries.append(entry)

columns = ['Dataset', 'Run Count', 'Model Version', 'Threading Enabled', 
           'Nanoseconds', 'Milliseconds', 'Seconds', 'Minutes']

_file_path: str = os.path.join(base_path, os.getenv('OUTPUT_EXCEL_NAME_WITH_ANNOTATIONS_MODIFIED'))
check = os.path.exists(_file_path)

old_time_consuption_data: pd.DataFrame = None
if check:
        try:
                old_time_consuption_data = pd.read_excel(_file_path, SHEET_NAME)
        except ValueError:
                check = False
           
time_consumption_data: pd.DataFrame = pd.DataFrame(entries, columns=columns)

if check and not old_time_consuption_data.empty:
        # Checking if in the excel is already the data. Case: this code is executed twice for the same data
        old_time_consuption_data = old_time_consuption_data.dropna()
        for idx in range(len(old_time_consuption_data)):
                condition = time_consumption_data[
                        (time_consumption_data['Dataset'] == old_time_consuption_data.iat[idx, 0]) & 
                        (time_consumption_data['Run Count'] == old_time_consuption_data.iat[idx, 1])
                ].index
                time_consumption_data = time_consumption_data.drop(condition).reset_index(drop=True)
        time_consumption_data = pd.concat([old_time_consuption_data, time_consumption_data]).reset_index(drop=True)

In [ ]:
def formatter_time(wb: Workbook, sheet_name: str): 
    ws = wb[sheet_name]
    header_font = Font(size=14, bold=True)
    for cell in ws["1:1"]:
        cell.font = header_font
    
    ADDITIONAL_LENGTH: int = 0
    ADJUSTED_WIDTH: int = 0
    MAX_LEN: int = 0
    for col in ws.iter_cols(min_row=1, max_row=1):
        for cell in col:
            MAX_LEN = len(str(cell.value))
            ADDITIONAL_LENGTH = (MAX_LEN + 2)
            ADJUSTED_WIDTH = 0
            ADJUSTED_WIDTH =  ADDITIONAL_LENGTH * 1.5
            ws.column_dimensions[utils_get_column_letter(cell.column)].width = ADJUSTED_WIDTH
            
    alignment = Alignment(vertical='center', horizontal='left')
    for row in ws.iter_rows():
        for cell in row:
            cell.alignment = alignment

    num_columns = ws.max_column
    header_range = f"A1:{utils_get_column_letter(num_columns)}1"
    ws.auto_filter.ref = header_range
    ws.freeze_panes = ws['A2']

    wrap_alignment = Alignment(wrap_text=True, vertical='top', horizontal='left')
    for row in ws.iter_rows(min_row=2):
        for cell in row:
            cell.alignment = wrap_alignment

In [ ]:
save_to_excel(time_consumption_data, formatter_time, SHEET_NAME, os.getenv("OUTPUT_EXCEL_NAME_WITH_ANNOTATIONS"))